In [3]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

In [2]:
load_dotenv()

pg_user = os.getenv('PG_USER')
pg_password = os.getenv('PG_PASSWORD')
pg_host = os.getenv('PG_HOST')
pg_port = os.getenv('PG_PORT')
pg_db = os.getenv('PG_GPU_DB')

pg_conn_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
pg_engine = create_engine(pg_conn_str)

Descriptive Analytics

Business Question:

In [ ]:
sql_query = '''
SELECT ...
FROM ...
WHERE ...
GROUP BY ...
'''
df = pd.read_sql(sql_query, pg_conn_str)

Diagnostic Analytics

Business Question

In [ ]:
sql_query = '''
SELECT ...
FROM ...
WHERE ...
GROUP BY ...
'''
df = pd.read_sql(sql_query, pg_conn_str)